In [ ]:
import os.path as path

if not path.exists("/content/drive"):
  !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !sudo apt-get update -qq 2>&1 > /dev/null
  !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
  !google-drive-ocamlfuse

  !sudo apt-get install -qq w3m # to act as web browser 
  !xdg-settings set default-web-browser w3m.desktop # to set default browser
  %cd /content
  !mkdir drive
  %cd drive
  !mkdir MyDrive
  %cd ..
  %cd ..
  !google-drive-ocamlfuse /content/drive/MyDrive

W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf

# Load Dependencies

In [ ]:
! pip install -q transformers
! pip install -q deep-phonemizer
! pip install -q wandb

     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 86.3 MB/s 
     |████████████████████████████████| 182 kB 86.0 MB/s 
     |████████████████████████████████| 7.6 MB 81.1 MB/s 
     |████████████████████████████████| 1.9 MB 8.3 MB/s 
     |████████████████████████████████| 168 kB 78.8 MB/s 
     |████████████████████████████████| 182 kB 85.7 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 97.5 MB/s 
     |████████████████████████████████| 166 kB 98.9 MB/s 
     |████████████████████████████████| 166 kB 85.5 MB/s 
     |████████████████████████████████| 162 kB 93.4 MB/s 
     |████████████████████████████████| 162 kB 79.0 MB/s 
     |████████████████████████████████| 158 kB 99.3 MB/s 
     |████████████████████████████████| 157 kB 99.8 MB/s 
     |████████████████████████████████| 157 kB 89.2 MB/s 
     |████████████████████████████████| 157 kB 100.3 MB/s 
     |███████████

# Imports

In [ ]:
import numpy as np
import pandas as pd 

import torch
from transformers import (GPT2Tokenizer, 
                          GPT2LMHeadModel, 
                          GPT2Config, 
                          AdamW, 
                          get_linear_schedule_with_warmup)

from torch.utils.data import (Dataset, 
                              random_split,
                              DataLoader,
                              RandomSampler,
                              SequentialSampler)

import torch.nn as nn
import os
from os import listdir
from os.path import isfile, join
import subprocess
import wandb

wandb.login(key="0eb6ccf942b6f35c3ce2c7debc54049c68ed8251")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
path_to_project_folder = ""

if not os.path.isdir("./Project"):
    env = os.environ.copy()
    subprocess.run(
        'ln -s "/content/drive/MyDrive/IDL/Project" /content/Project',
        shell=True,
        env=env,
    )

In [ ]:
! curl https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/DeepPhonemizer/en_us_cmudict_forward.pt --output en_us_cmudict_forward.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.6M  100 63.6M    0     0  13.0M      0  0:00:04  0:00:04 --:--:-- 15.8M


# Hyperparameter Config

In [ ]:
config = {
    "path_to_data_folder": '/content/Project/data/',
    'random_seed': 73,
    'batch_size': 1,
    'max_len': 64,
    'limerick_file_name': 'limericks_ballas_oedilf_clean.csv',
    'model_name': 'gpt2-medium',
    'training_storage_path': '/content/Project/trainings/',
    'iteration_step_to_log_generation': 1500,
    'iteration_step_to_log_checkpoint': 40000,
    'total_iterations': 154000,
    'learning_rate': 1e-4,
    'eps': 1e-8,
    'warmup_steps': 10000
}

# Data Loading and model config

In [ ]:
limerick_df = pd.read_csv(os.path.join(config['path_to_data_folder'], config['limerick_file_name']), index_col=0)
limerick_df = limerick_df.fillna('')
len(limerick_df)

153797

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(config['model_name'], )
tokenizer.model_max_length = config['max_len']
tokenizer.add_tokens('\n')

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
print(tokenizer.model_max_length)

64


In [ ]:
class LimerickDataset(Dataset):
    def __init__(self, data, max_length=config['max_len']):
        self.input_ids = []
        self.attn_masks = []
        
        for limerick in data:
            encodings_dict = tokenizer('<BOS>' + limerick + '<EOS>',
                                     truncation=False,
                                     max_length=max_length,
                                     padding='max_length'
                                    )

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
limerick_dataset = LimerickDataset(limerick_df['limerick'].values, max_length=config['max_len'])

KeyError: ignored

In [ ]:
limerick_dataloader = DataLoader(limerick_dataset, sampler=RandomSampler(limerick_dataset), batch_size=config['batch_size'])

In [ ]:
# GPT Configuration
configuration = GPT2Config(
        vocab_size=len(tokenizer), 
        n_positions=config['max_len']).from_pretrained(config['model_name'], output_hidden_states=True, bos_token_id=50258, eos_token_id=50259, pad_token_id=50260)

# Model Definition
model = GPT2LMHeadModel.from_pretrained(config['model_name'], config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Optimizer
optimizer = AdamW(model.parameters(), lr=config['learning_rate'], eps=config['eps'])
total_steps = len(limerick_dataloader) * 1

# Scheduler
lr_scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=config['warmup_steps'],
                                            num_training_steps=config['total_iterations'])
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
run = wandb.init(
    name = "GPT_CrossEntropy_base2", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "test-project" ### Project should be created in your wandb account
)

In [ ]:
prompt = "<BOS>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)


def log_generation(sample_outputs, iter_no):
    with open('generation_log.txt', 'a') as log_file:
        log_file.write(f"-- Iteration {iter_no} -- \n\n")
        print(f"\n\n -- Iteration {iter_no} --")
        for i, sample_output in enumerate(sample_outputs):
            log_limerick = "{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True))
            log_file.write(log_limerick)
            print(log_limerick)

def log_checkpoint(iteration, model, optimizer, lr_scheduler, metric=None):
    if iteration % config['iteration_step_to_log_checkpoint'] == 0 or iteration == config['total_iterations']:
        state = {
            'iteration': iteration + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(), 
            'lr_scheduler_state_dict': lr_scheduler.state_dict()
        }
        check_point_dir = config['training_storage_path']

        if not os.path.exists(check_point_dir):
            os.makedirs(check_point_dir)

        if metric == None:
            checkpoint_file_path = check_point_dir + f"/poet_ai_checkpoint_base2.h5"
            torch.save(state, checkpoint_file_path)
        else:
            # considering minimization effort
            onlyfile_metrics = [float(f.split("_checkpoint.h5")[0]) for f in listdir(check_point_dir) if isfile(join(check_point_dir, f)) and "_checkpoint.h5" in f]

            if len(onlyfile_metrics) > 0 and metric < sorted(onlyfile_metrics)[0]:
                checkpoint_file_path = check_point_dir + f"/{metric}_checkpoint.h5"
                torch.save(state, checkpoint_file_path)
                os.remove(check_point_dir + f"/{sorted(onlyfile_metrics)[0]}_checkpoint.h5")

def generate(iteration, model, optimizer, lr_scheduler):
    if iteration % config['iteration_step_to_log_generation'] == 0 or iteration == config['total_iterations']:
        sample_outputs = model.generate(generated, 
                                        do_sample=True,   
                                        top_k=50, 
                                        max_length=config['max_len'],
                                        top_p=0.95, 
                                        num_return_sequences=5)
        
        log_generation(sample_outputs, iteration)

In [ ]:
prompt = "<BOS> There was once a big mountain"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
output = model(generated)
sample_outputs = model.generate(generated, 
                                        do_sample=True,   
                                        top_k=50, 
                                        max_length=config['max_len'],
                                        top_p=0.95, 
                                        num_return_sequences=5, output_scores=True)
for i, sample_output in enumerate(sample_outputs):
            log_limerick = "{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True))
            print(log_limerick)

0: There was once a big mountain 
 but weve learned to accept mountains of pillage 
 the last was a crash 
 but the first was a flash 
 on a flash we could see it from glafal


1: There was once a big mountain 
 where a town did its part all the sallying 
 of the troops it has shown 
 that its troops may well go 
 bout an hour of time or theyre going


2: There was once a big mountain 
 with an alliname fame that you see 
 it was known as cunei 
 though it might seem obtuse 
 its so little theres none in a tree


3: There was once a big mountain 
 the mountains of nile are still tall 
 theres a great beach on top 
 and you might even drop 
 a few steps in its glacial tall wall


4: There was once a big mountain 
 for mountain is now just a cramp 
 this is plain to the eye 
 so be sure to stay clear 
 your new mount and your mount and all yap




In [ ]:
output.logits.shape

torch.Size([1, 5, 50261])

In [ ]:
sample_outputs.sequences

AttributeError: ignored

In [ ]:
def train(num_iterations):
    while (num_iterations < config['total_iterations']):

        print(f'Iteration {num_iterations} of', config['total_iterations'])
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(limerick_dataloader):
            optimizer.zero_grad()
            b_input_ids = batch[0].to(device)
            b_labels = batch[0].to(device)
            b_masks = batch[1].to(device)

            outputs = model(b_input_ids,
                            labels=b_labels,
                            attention_mask=b_masks,
                            token_type_ids=None)

            loss = outputs[0]

            batch_loss = loss.item()
            total_train_loss += batch_loss

            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            print(f'Iteration number: {num_iterations}')
            wandb.log({"train_loss": loss.item(),'train_iterations': num_iterations})
            generate(iteration=num_iterations, model=model, optimizer=optimizer, lr_scheduler=lr_scheduler)
            log_checkpoint(num_iterations, model, optimizer, lr_scheduler)
            
            num_iterations += 1

In [ ]:
train(0)

Streaming output truncated to the last 5000 lines.
Iteration number: 160936
Iteration number: 160937
Iteration number: 160938
Iteration number: 160939
Iteration number: 160940
Iteration number: 160941
Iteration number: 160942
Iteration number: 160943
Iteration number: 160944
Iteration number: 160945
Iteration number: 160946
Iteration number: 160947
Iteration number: 160948
Iteration number: 160949
Iteration number: 160950
Iteration number: 160951
Iteration number: 160952
Iteration number: 160953
Iteration number: 160954
Iteration number: 160955
Iteration number: 160956
Iteration number: 160957
Iteration number: 160958
Iteration number: 160959
Iteration number: 160960
Iteration number: 160961
Iteration number: 160962
Iteration number: 160963
Iteration number: 160964
Iteration number: 160965
Iteration number: 160966
Iteration number: 160967
Iteration number: 160968
Iteration number: 160969
Iteration number: 160970
Iteration number: 160971
Iteration number: 160972
Iteration number: 160973

In [ ]:
run.finish()

train_iterations,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▇▆▄▆▃▄▃▃▅▃▅▄▇▄▅▃▂▂▄▅▂▃▂▃▃▃▃▃▄▂▃▃▂▃▂▄▂▁▂
train_iterations,165821
train_loss,2.19574


In [ ]:
import csv
prompt = "<BOS>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
output = model(generated)


for i in range (201):
  sample_outputs = model.generate(generated, 
                                        do_sample=True,   
                                        top_k=50, 
                                        max_length=config['max_len'],
                                        top_p=0.95, 
                                        num_return_sequences=5, output_scores=True)

  for i, sample_output in enumerate(sample_outputs):
            log_limerick = "{}\n\n".format(tokenizer.decode(sample_output, skip_special_tokens=True))
            with open('/content/Project/generated_1000.csv','a', newline='') as log_file:
              a = csv.writer(log_file,delimiter=',', quoting=csv.QUOTE_MINIMAL)
              a.writerow([log_limerick])

In [ ]:
!ls

bin	 en_us_cmudict_forward.pt  lib	  opt	      sbin		 tools
boot	 etc			   lib32  proc	      srv		 usr
content  generation_log_1000.txt   lib64  python-apt  sys		 var
datalab  generation_log.txt	   media  root	      tensorflow-1.15.2  wandb
dev	 home			   mnt	  run	      tmp
